# <font color='gold'> ***Análise Exploratória: Incêndios Florestais Brasileiros*** </font> 🔥🌳

# <font color='gold'> ***Objetivos da Análise*** 🎯 </font>
---

Nesse projeto, realizaremos uma Analise Exploratória de Dados (EDA) em um [banco de dados](https://www.kaggle.com/datasets/gustavomodelli/forest-fires-in-brazil) que contém uma série histórica de incêndios florestais no Brasil em diferentes estados. Mais especificamente, desse projeto iremos:

- 🎲 Realizar uma análise descritiva das características principais dos dados, como suas dimensões, seus registros e tipos de dados, valores nulos e outliers, etc;

- 🧹 Limpar e tratar os dados;

- 📊 Utilizar de visualizações gráficas para geração de insights e análises mais aprofundadas acerca dos incêndios e suas características.

Como questões centrais que abordaremos nessa pesquisa, temos:

>***1. Quais os estados e regiões mais afetados por incêndios durante toda a série histórica?***
>
>***2. Durante qual período de qual governante brasileiro tivemos mais incêndios no Brasil? e qual o menor valor?***

Entre outras questões que serão abordadas durante o projeto.

# <font color='gold'>***Importando Pacotes e Conhecendo Dados*** 📦🎲</font>
---

In [842]:
# Importando bibliotecas necessárias para a análise:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Definindo parâmetros:

# Setando estilo escuro do grid para os gráficos.
sns.set_theme(context='notebook', style='darkgrid')

# Utilizando a biblioteca warnings para retirar avisos da execução do código
# para uma visualização mais limpa.
warnings.filterwarnings('ignore')

# Mostrando versões utilizadas dos pacotes:
print('Pacotes Prontos! Versões Utilizadas:\n')
print(f'Pandas      --> {pd.__version__}')
print(f'Numpy       --> {np.__version__}')
print(f'Matplotlib  --> {matplotlib.__version__}')
print(f'Seaborn     --> {sns.__version__}')

Pacotes Prontos! Versões Utilizadas:

Pandas      --> 1.4.2
Numpy       --> 1.21.5
Matplotlib  --> 3.6.2
Seaborn     --> 0.11.2


In [843]:
# Criando o Dataframe:
data = pd.read_csv('data/amazon.csv', sep=',', encoding='latin-1')

<font color='gold'> ⚠️**Nota** </font>

>A seleção do encoding como [latin-1](https://www.ic.unicamp.br/~stolfi/EXPORT/www/ISO-8859-1-Encoding.html) foi utilizada pois existem caracteres especiais nos dados, assim, a definição do enconding específico foi necessária (em vez do padrão UTF-8).

In [844]:
# Visualizando primeiros 5 registros dos dados:
data.head()

,year,state,month,number,date
0,1998,Acre,Janeiro,0.0,1998-01-01
1,1999,Acre,Janeiro,0.0,1999-01-01
2,2000,Acre,Janeiro,0.0,2000-01-01
3,2001,Acre,Janeiro,0.0,2001-01-01
4,2002,Acre,Janeiro,0.0,2002-01-01


In [845]:
# Visualizando uma sample aleatória dos dados:
data.sample(5)

,year,state,month,number,date
5976,1998,Sergipe,Janeiro,0.0,1998-01-01
2444,2011,Mato Grosso,Março,252.0,2011-01-01
447,2005,Alagoas,Novembro,25.0,2005-01-01
2264,2010,Maranhao,Junho,494.0,2010-01-01
2176,2002,Maranhao,Fevereiro,116.0,2002-01-01


Para termos uma compreensão clara de nossos dados, iremos criar um dicionário de variáveis contendo as colunas do Dataset e a descrição de seu conteúdo:

| Variável   | Conteúdo                                 |
|------------|------------------------------------------|
| `year`     | Ano em que o incêndio florestal ocorreu  |
| `state`    | Estado brasileiro                        |
| `month`    | Mês em que o incêndio ocorreu            |
| `number`   | Quantidade de incêndios reportados       |
| `date`     | Data em que o incêndio foi reportado     |

Com alguns dos registros aparentes, uma característica notável é a aparente duplicata nas colunas. 

- A coluna `year`, que representa o ano em que o incêndio ocorreu e a coluna `date` que informa a data em que o incêndio foi reportado, aparentam possuir os mesmos dados:

    - Quando a coluna year apresenta o ano de 1998, a coluna date informa a data 1998-01-01, e esse padrão da coluna date apenas repetir a coluna year com a adição das datas "1 de Janeiro" se mostra presente em todo dataset como mostrado pelo uso do comando `data.sample(5)`.

Dessa forma, iremos arranjar essas informações de uma maneira diferente:

- Substituir os valores da coluna `month` por numéricos ('Janeiro' = 1, 'Fevereiro' = 2, etc);

- Utilizar a coluna `date` para mostrar o ano e o mês (agora em formato numérico) do registro;

- Transformar essa nova coluna `date` para datetime.

Como outros ajustes, também iremos renomear as colunas para que tenham a primeira letra maiúscula e também, renomear algumas colunas. Também iremos inserir algumas novas colunas para fortalecer as análises, tendo essas etapas ocorrendo na etapa de limpeza e tratamento dos dados.

Para uma visão mais completa das dimensões dos dados que estamos trabalhando, vamos checar o shape do Dataset:

In [846]:
# Checando dimensões dos dados:
data.shape

(6454, 5)

Temos ***6454 registros*** em ***5 variáveis***.

Também se faz necessária a checagem do tipo de dados dessas variáveis:

In [847]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    6454 non-null   int64  
 1   state   6454 non-null   object 
 2   month   6454 non-null   object 
 3   number  6454 non-null   float64
 4   date    6454 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 252.2+ KB


Com essa visualização, temos algumas informações importantes sobre os dados:

- O dataset não possui null values;

- As colunas `year`, `month` e `date` serão unidas numa só coluna datetime;

- Os dados alocam pouca quantidade de memória (cerca de 250 Kb).

Para coletarmos mais informações sobre possíveis outliers e também sobre a distribuição dos dados, utilizaremos o método `.describe()`:

In [848]:
# Descrição estatística dos dados (colunas numéricas):
data.describe().T

,count,mean,std,min,25%,50%,75%,max
year,6454.0,2007.461729,5.746654,1998.0,2002.0,2007.0,2012.0,2017.0
number,6454.0,108.293163,190.812242,0.0,3.0,24.0,113.0,998.0


Com a descrição estatística das colunas numéricas do dataset, podemos notar:

- Até 75% dos dados (terceiro quantil) relacionados ao número de incêndios relatados vão até 113, já o valor máximo é de 998, um outlier claro.

Nesse caso, outliers são esperados como "normais", pois se tratando de número de focos de incêndios florestais num país continental como o Brasil, não existe uma regra ou uma lógica exata que dita o intervalo desses números, tendo muitas variávies que os influenciam (ação humana mais intensa ou menos intensa numa determinada região num determinado período de tempo, uma estação do ano muito mais fria ou quente do que o normal, etc).

Sendo assim, os registros de número de focos de incêndio não serão alterados, pois buscamos ter uma visão completa desses dados.

Também iremos coletar informações sobre as colunas categóricas do dataset:

In [849]:
# Descrição estatística dos dados (colunas categóricas):
data.describe(exclude='number').T

,count,unique,top,freq
state,6454,23,Rio,717
month,6454,12,Janeiro,541
date,6454,20,1998-01-01,324


Com isso, temos a informação de que o estado que mais vezes consta no dataset é o Rio de Janeiro, com 717 registros diferentes de focos de incêndio.

Também temos a informação de que o mês mais comum nos dados é Janeiro.

É importante termos em mente que isso não necessariamente significa que o estado do Rio de Janeiro e o mês de Janeiro são os que mais possuem focos de incêndio no geral. Apenas estamos indicando que essas duas informações são as mais presentes nos dados, sem considerarmos a dimensão e quantidade dos focos de incêndio atribuidos a eles.

# <font color='gold'> ***Limpeza e Tratamento*** 🗃️ </font>

Nessa etapa, iremos realizar todas as modificações e tratamentos que descrevemos anteriormente. Começando pelo refatoramento das colunas `year`, `date` e `month`. Para isso, iremos primeiramente definir uma função para iterar sobre nossos dados e passar os meses de linhas de texto para números.

## **Reformulando formato de data**:

In [850]:
def month_string_to_int(df: pd.DataFrame) -> pd.DataFrame:
    '''Essa função pega um Dataframe fornecido pelo usuário e altera
    os valores dos meses de linhas de texto para números.

    Utilizando uma função lambda, checamos cada string dos meses, onde cada
    uma delas recebe um número equivalente ao mês do ano.
    '''
    df['month'] = df['month'].apply(
        lambda month_id:
        1 if month_id == 'Janeiro' else
        2 if month_id == 'Fevereiro' else
        3 if month_id == 'Março' else
        4 if month_id == 'Abril' else
        5 if month_id == 'Maio' else
        6 if month_id == 'Junho' else
        7 if month_id == 'Julho' else
        8 if month_id == 'Agosto' else
        9 if month_id == 'Setembro' else
        10 if month_id == 'Outubro' else
        11 if month_id == 'Novembro' else
        12 if month_id == 'Dezembro' else
        month_id
    )   


Antes de iniciarmos as modificações nos dados, é uma boa prática criar uma cópia dos mesmos, assim se preservam os dados no formato original e se cria outra versão para a limpeza e manipulação:

In [851]:
# Criando cópia dos dados para limpeza:
data_clean = data.copy()

In [852]:
# Aplicando a função no dataset:
data_clean == month_string_to_int(data_clean)

# Exibindo novos resultados:
data_clean.sample(5)

,year,state,month,number,date
3231,2001,Pará,7,708.000,2001-01-01
2963,2012,Minas Gerais,5,144.000,2012-01-01
6410,2013,Tocantins,10,2.155,2013-01-01
3776,2008,Paraiba,10,78.000,2008-01-01
1486,2009,Distrito Federal,3,0.000,2009-01-01


Agora que os meses estão no formato numérico (ainda com o dtype string, porém com valores numéricos), vamos criar um método para unificar a coluna `month` com a coluna `year`, para termos o ano e o mês do incêndio florestal na mesma coluna:

In [853]:
# Definindo quais as colunas a serem manipuladas:
columns = ['year', 'month']

# Com uma função lambda, unificaremos os valores da coluna 'year' e 'month'
# utilizando o caracter '-', retornando assim a data no formato 'ano-mês':
data_clean['date'] = data_clean[columns].apply(
    lambda new_date: 
    '-'.join(new_date.values.astype(str)), axis=1
)

In [854]:
# Checando resultados:
data_clean.sample(5)

,year,state,month,number,date
6025,2007,Sergipe,3,2.0,2007-3
64,2002,Acre,4,0.0,2002-4
5436,2015,Roraima,9,77.0,2015-9
6274,2017,Tocantins,3,98.0,2017-3
1757,2001,Espirito Santo,5,8.0,2001-5


Agora que temos a coluna `date` constando o ano e o mês do incêndio florestal, basta atribuir o data type `datetime` à ela:

In [855]:
# Transformando a coluna date para datetime:
data_clean['date'] = pd.to_datetime(data_clean['date'])

# Checando resultados:
data_clean.sample(5)

,year,state,month,number,date
3982,2015,Pernambuco,8,21.000,2015-08-01
6440,2003,Tocantins,12,109.000,2003-12-01
2583,2010,Mato Grosso,10,4.465,2010-10-01
1850,2014,Espirito Santo,9,61.000,2014-09-01
1061,2002,Bahia,6,118.000,2002-06-01


In [856]:
# Checando dtypes:
data_clean.dtypes

year               int64
state             object
month              int64
number           float64
date      datetime64[ns]
dtype: object

Com essa nova coluna `date`, temos a informação das datas dos incêndios florestais no formato apropriado. Agora, vamos dropar as colunas `year` e `month`:

In [857]:
# Excluindo coluna month:
data_clean = data_clean.drop(['month', 'year'], axis=1)

In [858]:
# Checando novo formato dos dados:
data_clean.sample(5)

,state,number,date
3080,Minas Gerais,246.000,2009-11-01
2074,Goias,1.898,1999-09-01
1555,Distrito Federal,17.000,1998-07-01
5713,Santa Catarina,84.000,2013-11-01
1901,Espirito Santo,5.000,2005-12-01


## **Criando novas variáveis**

Com os dados no formato atual, já se é possível realizar algumas análises, porém, visamos nesse projeto uma visão mais aprofundada dos incêndios florestais no Brasil, e por essa razão se faz necessária a criação de algumas novas variáveis para nossos dados, sendo elas:

- Coluna `UF` > A coluna que representará a Unidade da Federação do estado (SP, RJ, BA, etc). Essa coluna será criada pois os nomes dos estados não podem ser confiados de estarem no formato adequado para criação de Dashboards e análises, como exemplo, o estado do Rio de Janeiro apenas consta com o nome 'Rio' no dataset. Sendo assim, a criação da coluna UF vem como forma de padronizar esses indicadores de estado.

- Coluna `Region` > A coluna que representará a região brasileira na qual o estado pertence (Norte, Sul, Sudeste, etc). Essa coluna será concebida como forma de aprofundar as análises posteriores, nos possibilitando ter respostas acerca de quais regiões foram mais e menos afetadas pelos incêndios.

- Coluna `President` > A coluna que representará o nome de qual mandatário brasileiro estava em vigência quando os incêndios ocorreram. Essa coluna nos permitirá ter um olhar mais histórico acerca desses registros, onde poderemos comparar as diferentes gestões dos diversos mandatários nesses períodos e identificar quais passaram por um período de mais ou menos incêndios.

### Criando coluna: UF

Para a criação da coluna `UF`, iremos criar uma função que percorre os dados e atribui a UF correta para seu respectivo estado, para isso, vamos checar quais são os estados presentes no Dataset:

In [859]:
# Checando quais estados aparecem nos dados:
states = list(data_clean['state'].unique())

for (index, names) in enumerate(states, start=1):
    print(f'Estado {index:2} --> {names}')

Estado  1 --> Acre
Estado  2 --> Alagoas
Estado  3 --> Amapa
Estado  4 --> Amazonas
Estado  5 --> Bahia
Estado  6 --> Ceara
Estado  7 --> Distrito Federal
Estado  8 --> Espirito Santo
Estado  9 --> Goias
Estado 10 --> Maranhao
Estado 11 --> Mato Grosso
Estado 12 --> Minas Gerais
Estado 13 --> Pará
Estado 14 --> Paraiba
Estado 15 --> Pernambuco
Estado 16 --> Piau
Estado 17 --> Rio
Estado 18 --> Rondonia
Estado 19 --> Roraima
Estado 20 --> Santa Catarina
Estado 21 --> Sao Paulo
Estado 22 --> Sergipe
Estado 23 --> Tocantins


É aparente que nem todos os estados da nação estão presentes nos dados. Temos 23 estados diferentes porém o Brasil possuí 27 unidades federativas. 

Outro fator relevante é o registro errado do estado de Piauí, que consta apenas como 'Piau'. Esse registro errado apenas fortalece o motivo da qual estamos criando a coluna UF, para padronizar esses dados e garantir que todos estão corretamente representados.

Antes de definir nossa função de criação das UFs, vamos corrigir essa entrada do estado de Piauí e também adicionar o nome completo do estado do Rio de Janeiro:

In [860]:
# Corrigindo registros de Rio de Janeiro e Piauí:
data_clean['state'] = (
    data_clean['state']
    .replace(['Rio', 'Piau'], ['Rio de Janeiro', 'Piaui'])
)

# Checando os novos registros:
data_clean.query('state == "Rio de Janeiro" | state == "Piaui"').sample(5)

,state,number,date
4763,Rio de Janeiro,29.0,1999-12-01
4494,Rio de Janeiro,18.0,2009-10-01
4961,Rio de Janeiro,169.0,1998-10-01
4541,Rio de Janeiro,5.0,2016-12-01
4831,Rio de Janeiro,38.0,2008-03-01


Com os estados presentes nos dados de maneira correta, podemos definir nossa função de atribuição das UFs:

In [861]:
def set_uf(df: pd.DataFrame) -> pd.DataFrame:
    '''Essa função recebe um dataframe, itera sobre os registros e
    atribui aos estados suas respectivas UFs.
    
    Para isso, criamos um dicionário que contém o par 'estado: UF' e 
    e atribuímos diretamente esses valores a nova coluna.
    '''
    state_uf = {
        'Acre': 'AC',
        'Alagoas': 'AL',
        'Amapa': 'AP',
        'Amazonas': 'AM',
        'Bahia': 'BA',
        'Ceara': 'CE',
        'Distrito Federal': 'DF',
        'Espirito Santo': 'ES',
        'Goias': 'GO',
        'Maranhao': 'MA',
        'Mato Grosso': 'MT',
        'Minas Gerais': 'MG',
        'Pará': 'PA',
        'Paraiba': 'PB',
        'Pernambuco': 'PE',
        'Piaui': 'PI',
        'Rio de Janeiro': 'RJ',
        'Rondonia': 'RO',
        'Roraima': 'RR',
        'Santa Catarina': 'SC',
        'Sao Paulo': 'SP',
        'Sergipe': 'SE',
        'Tocantins': 'TO'
    }

    df['UF'] = df['state'].map(state_uf)

    return df

In [862]:
# Aplicando a função:
data_clean = set_uf(data_clean)

# Exibindo resultados:
data_clean.sample(5)

,state,number,date,UF
1739,Espirito Santo,22.0,2003-04-01,ES
1028,Bahia,40.0,2009-04-01,BA
5498,Santa Catarina,0.0,1998-01-01,SC
1885,Espirito Santo,18.0,2009-11-01,ES
4930,Rio de Janeiro,258.0,2007-08-01,RJ


### Criando coluna: Region

Para adicionar as regiões de cada estado, vamos criar uma função que itera sobre os registros e atribui a cada estado sua devida região:

In [863]:
def set_region(df: pd.DataFrame) -> pd.DataFrame:
    '''Essa função adiciona as regiões dos respectivos
    estados com base em sua UF.
    
    Delimita-se uma lista com os estados para cada região e utiliza-se
    uma função lambda para aplicar ao dataset.
    '''
    regiao_norte = ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO']
    regiao_nordeste = ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'SE']
    regiao_centro_oeste = ['DF', 'GO', 'MT']
    regiao_sudeste = ['ES', 'MG', 'RJ', 'SP']
    regiao_sul = ['SC']

    df['Region'] = df['UF'].apply(
        lambda state:
        'Norte' if state in regiao_norte else
        'Nordeste' if state in regiao_nordeste else
        'Centro Oeste' if state in regiao_centro_oeste else
        'Sudeste' if state in regiao_sudeste else
        'Sul' if state in regiao_sul else
        state
    )

    return df

In [864]:
# Aplicando a função:
data_clean = set_region(data_clean)

# Checando novos dados:
data_clean.sample(5)

,state,number,date,UF,Region
3422,Paraiba,6.000,2013-04-01,PB,Nordeste
3115,Pará,1.768,2005-01-01,PA,Norte
1375,Ceara,505.000,2017-09-01,CE,Nordeste
818,Amazonas,2.000,1998-06-01,AM,Norte
2904,Minas Gerais,143.000,2013-02-01,MG,Sudeste


### Criando coluna: President

Para podermos inserir nos dados o mandatário vigente durante os registros de incêndio florestal, primeiro iremos checar o intervalo de tempo que temos nos dados:

In [865]:
# Checando intervalo de anos:
print(f'Ano mínimo: {data_clean.date.min().year}')
print(f'Ano máximo: {data_clean.date.max().year}')

Ano mínimo: 1998
Ano máximo: 2017


Com os anos definidos, utilizando uma [lista com todos os mandatários](https://pt.wikipedia.org/wiki/Lista_de_presidentes_do_Brasil) do Brasil, iremos atribuí-los aos respectivos períodos que constam nos dados. Para isso, utilizaremos uma função:

In [866]:
def set_president(df: pd.DataFrame) -> pd.DataFrame:
    '''Essa função itera sobre os anos do dataset e atribuí a uma nova coluna
    o nome do mandatário vigente durante esse período.
    '''
    df['President'] = df['date'].apply(
        lambda date:
        'FHC' if (date.strftime('%Y-%M') > '1997-01' and 
                    date.strftime('%Y-%M') < '2002-12') else
        'Lula' if (date.strftime('%Y-%M') > '2002-12' and 
                    date.strftime('%Y-%M') < '2010-12') else
        'Dilma' if (date.strftime('%Y-%M') > '2010-12' and 
                    date.strftime('%Y-%M') < '2016-12') else
        'Temer' if (date.strftime('%Y-%M') > '2016-12') else
        date
    )

    return df

In [867]:
# Aplicando a função:
data_clean = set_president(data_clean)

# Visualizando novos resultados:
data_clean.sample(5)

,state,number,date,UF,Region,President
5576,Santa Catarina,36.0,2016-04-01,SC,Sul,Dilma
109,Acre,4.0,2007-06-01,AC,Norte,Lula
551,Amapa,2.0,2010-04-01,AP,Norte,Lula
2021,Goias,70.0,2006-06-01,GO,Centro Oeste,Lula
4070,Piaui,42.0,2004-01-01,PI,Nordeste,Lula


Com todas as colunas necessárias criadas para as análises, vamos realizar os últimos ajustes nos dados, isto é, renomear as colunas e reposicioná-las no dataset:

### Renomeando e Reposicionando Colunas

In [868]:
# Renomeando colunas utilizando um dicionário:
data_clean = data_clean.rename(columns={
    'state': 'State',
    'number': 'Num_wild_fires',
    'date': 'Date'
})

# Reposicionando colunas:
data_clean = data_clean.loc[
    :, 
    ['State', 'UF', 'Region', 
    'Num_wild_fires', 'Date', 'President']
]
# Visualizando novos dados:
data_clean.head()

,State,UF,Region,Num_wild_fires,Date,President
0,Acre,AC,Norte,0.0,1998-01-01,FHC
1,Acre,AC,Norte,0.0,1999-01-01,FHC
2,Acre,AC,Norte,0.0,2000-01-01,FHC
3,Acre,AC,Norte,0.0,2001-01-01,FHC
4,Acre,AC,Norte,0.0,2002-01-01,FHC


Com as colunas nomeadas e reorganizadas temos uma padronização melhor, onde elas descrevem bem o conteúdo das variáveis e também, iniciam-se com letra maiúscula.